# Tutorial 10 (in Wk 11):  Scraping and Working with Geo-Data

## Introduction
In this tutorial, you will get practical exposure to web scraping techniques and with geo-data.

# EXERCISE 1 - Collecting Geo-Data for Neighbourhoods
Similar to last week, we will try to extract some useful information from a website. This time it is about boundary data for the neighbourhoods which we are working on in the assignment.

We will use the following Python libraries:
- **Request**         for interacting with websites and web services
- **Beautiful Soup**  for webpage parsing

In the practical assignment, you are asked to combine several datasets about *neighbourhoods*.
While the statistical data from the ABS share the same neighbourhood ID, the example dataset about transport options - the carsharing pods - does not have this ID, nor do the names match between car pod locations and neighbourhoods.
The car pods do have a GPS location though...

**Background:** Car pods are dedicated parking places where car sharing cars are parked. There are different car sharing companies operating in Sydney, all of which follow the model that users have to book a car in advance, before they can drive it. Cars are parked at the dedicated pods, and have to be both collected and returned to those locations for each use.

In order to associate car pods with the other data on neighbourhoods, we need to collect some spatial data for the neighbourhoods too. This is the first exercise in this tutorial.

Our goal in this first exercise is to extract the saptial boundaries of each Sydney neighbourhood (to be precise: of each SA2-level area in Greater Sydney) from the ABS website: http://stat.abs.gov.au/

The ABS collects census data in different granularities. We will work with the Statistical Area 2 (SA2) data which roughly corresponds to neighbourhoods of one or two suburbs. The level on top would be SA3 data. For example, *Darlinghurst* is a SA2 area which lies within the wider SA3 area *Sydney Inner City*, which in turn belongs to the SA4 area *Sydney - City and Inner South*, which is part of the Greater Capital City Statistical Area (GCCSA) of *Greater Sydney* in the state of *New South Wales*.

You can check the location and boundary of the various ABS areas here (choose "2016 Statistical Area Level 2 (SA2)" in the selector):
http://stat.abs.gov.au/itt/r.jsp?ABSMaps

Let's start by having a look at the actual webpage.
The following code requests the webpage for SA2 area **Glebe - Forest Lodge** from the ABS website (the given URL scheme is specific for this website):

In [ ]:
import requests
webpage_source = requests.get("http://stat.abs.gov.au/itt/r.jsp?RegionSummary&region=117031331&dataset=ABS_REGIONAL_ASGS&geoconcept=REGION&measure=MEASURE&datasetASGS=ABS_REGIONAL_ASGS&datasetLGA=ABS_NRP9_LGA&regionLGA=REGION&regionASGS=REGION").text
print(webpage_source)

You see again the HTMl source code of the actual website. There are a lot of scripts and additional information included in this website this time. The ABS website certainly is more complex than the Convicts Records website which we studied last week.

Have a look at the following Python code. It shows how the BeautifulSoup library is used to parse the webpage source code and then demonstrates different navigational commands of that library to identify specific tags within a (complex) page:

If you open the webpage in a web browser, you will see that it contains besides the actual census data also a map - and on this map, the boundaries of the neighbourhood is visible (and clickable).

We need to extract the underlying vecxtor data from this webpage.

## Approach 1: Trying to scrap map boundary data from area-details webpage

What we would like to do in this exercise is to extract the geographic boundaries of a neighbourhood from this page.

When you scroll around the HTML code or use the **Inspect** menu function of your web browser (eg. Chrome), you will find that the map is associated with the following HTML element
> &lt;div class="rTemplateMap map" id="map" ... title="Map of Glebe - Forest Lodge"&gt; 

>    &lt;div id="map_root" class="container" style=":..."&gt;
 
which describes the map object on that page.

Lets use Python to extract just this part.
To do so, we use the Beautiful Soup library to search for an HTML tag with id="map", extract it from the page into a variable *mapdetails*, and print it out:

In [ ]:
from bs4 import BeautifulSoup

page_content = BeautifulSoup(webpage_source, 'html5lib')
mapdetails  = page_content.find(id="map")
print(mapdetails)

This looks already like some useful information, for example to extract the map's title and hence the name of the neighbourhood. But where is our boundary data?

A bit further down inside this **map** div-element,  there are the following graphics and multiple path elements:
> &lt;svg overflow="hidden" id="map_gc" ...&gt;

> &lt;g id="graphicsLayer0_layer" data-geometry-type="polygon" ...&gt;

> &lt;g&gt;&lt;/g&gt;

> &lt;path fill="rgb(255, 255, 255)" stroke="rgb(0,0,0)" ...

...

> &lt;path fill="rgb(204, 153, 153)" stroke="rgb(255,0,0)" ...


Notice that most polygons are drawn in black (stroke="rgb(0,0,0)") and only one in red (stroke="rgb(255,0,0)"). All of them correspond to different boundaries of neighborhoods on the map, but only the red one is the current one for which we retrieved this page. The others, which are drawn in black, are neighboring areas.

Let's have a look at these ones:

In [7]:
polygondetails  = page_content.find(id="map_gc")
print(polygondetails)

None


Ok, this seems only to work when actually using the website and using the mouse on the map...


For more documentation about functions available in BautifulSoup, see here:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

## Approach 2: Download SA2 Boundary Data
Another approach might be to attempt to download the boundary data directly from here:

http://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.001July%202016?OpenDocument#Data

For example, on this page, the following boundary data
**Statistical Area Level 2 (SA2) ASGS Ed 2016 Digital Boundaries MapInfo** is available directly in theree different formats: **ESRI shapfile Format**, **TAB File Format**, and **MapInfo Interchange Format**.

The problem here is that these are very complex, binary exchange formats for geo-spatial data which require special software to parse and further process...

In [ ]:
# TODO: download one of the boundary files from above's website
raise NotImplementedError

## Approach 3: Get Boundary Data using Web APIs

In the final approach, we are looking at some examples on how to access web APIs which are specifically provided for program to retrieve data. The advantage is that the data is well defined - no distracting HTML tags in between.

But the services uses two different formats - either JSON or XML.

For **JSON**, we will use the standard language support in Python and its **request** library.<br>
For **XML**, we will use the **lxml** parser library.

### Google Web API
The Google Wep-API allows to convert a location address to a GPS location (and some information more). The following example looks up the GPS location as well as the general boundary of the suburb 'Darlington' at "Darlington, New South Wales, Australia":

In [23]:
# Lookup of a given address via Google Wep-API:
import requests
import json
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
my_params= {'address': 'Darlington, New South Wales, Australia','language':'en'}
response = requests.get(base_url, params = my_params)
#if you want to see the raw response, comment-in the following line
#print(json.dumps(response.json(), indent=4, sort_keys=False))

results      = response.json()['results']
nhood_geo    = results[0]['geometry']['location']
nhood_bounds = results[0]['geometry']['bounds']
print(my_params['address'])
print("Location: ", nhood_geo['lat'], nhood_geo['lng'])
print("Boundary: ", json.dumps(nhood_bounds, indent=4))

Darlington,New South Wales,Australia
Location:  -33.890334 151.19661
Boundary:  {
    "northeast": {
        "lat": -33.8876833,
        "lng": 151.1988246
    },
    "southwest": {
        "lat": -33.8940654,
        "lng": 151.1876691
    }
}


## YOUR TASK: Retrieve Geo-Locations and Boundaries for given Neighbourhoods

 - Use the Google Web-Api to retrieve the *GPS boundary* of the neighbourhoods as listed in the assignment data in 'Neighbourhoods.csv' 
- For this you might need to inspect first how the JSON response is structured: Do hence first a pretty-print of the corresponding JSON response data.
 - Tip: If Google rejects your request because of a 'request limit' being reached, please try a few minutes later again. Do not try to access the API too frequently, otherwise you might need to register for an application key with Google.
 - Tip: if you want to see a map for a given GPS location, try: https://www.latlong.net/

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

# EXERCISE 2: Writing Scraped Data to a Database
Lets store the extracted boundaries in PostgreSQL together with the other data about neighbourhoods.

For the **PostgreSQL** database output, use the **psycopg2** library as we used already in Week 4.

In the next step, we are looking into how to store the scraped data directly into a PostgreSQL database. To do so, we copy the following two utility functions from Week 4 to connect to PostgreSQL and to execute an SQL command there from Python (actually, pgquery() is an improved version of pgexec() from Week 4).

Please edit in the following cell the variables **YOUR_UNIKEY** and **YOUR_PW** to match your own PostgreSQL login.

In [ ]:
import psycopg2
import psycopg2.extras

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = #'unikey'#'<your_unikey>'
    YOUR_PW     = #'abcd1234'#'<your_SID>'
    try: 
        conn = psycopg2.connect(host='soit-db-pro-2.ucc.usyd.edu.au',
                                database='y18s1d2001_'+YOUR_UNIKEY,
                                user='y18s1d2001_'+YOUR_UNIKEY, 
                                password=YOUR_PW) 
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

def pgquery( conn, sqlcmd, args=None, silent=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholder in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if silent != False:
                    print("success: " + silent)
            except Exception as e:
                if e.pgcode != None and not(silent):
                    print("db read error: ")
                print(e)
    return retval

## YOUR TASK: Store Neighbourhood Boundaries in Database
Your task is as follows:
* Extend the table with your neighbourhood data, which matches the CSV files from Canvas about assignment 1, so that it can also store the geo-boundaries of a neighbourhood.
* Insert/Update the extracted information of the neighbourhood boundaries in that table
* Tip: Postgresql has some specific geometry data types which can be helpful here: https://www.postgresql.org/docs/9.5/static/datatype-geometric.html

In [ ]:
# TODO: replace the content of this cell with your Python + SQL solution
raise NotImplementedError

# EXERCISE 3: Spatial Join between Carsharing Pods and Neighbourhoods

The next step is to do a spatial join between the car sharing locations and the neighbourhood boundaries.

## YOUR TASK: Spatial Join between carsharing pods and neighbourhoods
Your task is as follows:
* Check with the postgresql documentation on how to do a spatial join
* Query in SQL for each carsharing pod in which neighbourhood it is located; if you have the boundaries then you can check for containment; otherwise you might need to look for the shortest distance between the pod location and the centre of a neighbourhood.
* It might be a good idea to store the computed relationships between pods and neighbourhoods in the database too.
* Tip: PostgreSQL supports a number of spatial functions for points and other geometry types: https://www.postgresql.org/docs/9.5/static/functions-geometry.html 

In [ ]:
# TODO: replace the content of this cell with your Python + SQL solution
raise NotImplementedError

# References

Books:
- Seppe van den Broucke and Bart Baesens: "Practical Web Scraping for Data Science", Springer 2018. (available electroinically via USYD library)

# End of Tutorial. Many Thanks.